## Problem Statement

Given Temperature and Precipitation at 9 stations and river flow at one location, predict river flow one week later.

*Predictors/Features*:  Temperature (${^o}C$) and Precipitation (mm) at 9 stations; river flow ($m^{3}s^{-1}$) at given location.

*Predictand/Target*: River Flow ($m^{3}s^{-1}$) at the same location one week later

## Science Background

*Why would we expect to be able to predict river flow with precipitation, temperature, and the previous river flow? How are these variables related?*


* In the absence of new water input, the river flow will decrease slowly over time. 
* River flow has high autocorrelation over some timescale (??)
* Increased precipitation -> increased river flow
* River flow is an "integrated" response in time to precipation
* In regions with snow, temperature above freezing leads to melting snow and increased river flow during the melt season
* In the non-melt season, higher temperatures could leader to more evaportation and result in less river flow, but this is likely a small impact 
* All datasets could have a climate change related trend or changes in variability over time
* All likely have a seasonal cycle

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn import preprocessing

from keras.layers import Input, Dense, Activation
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
from keras import regularizers
from keras import initializers
from keras import optimizers
from keras.utils.vis_utils import plot_model

ModuleNotFoundError: No module named 'keras'

### My Functions -- move to utils

In [ ]:
def lasso(X,Y):

    # X_train[time,features]
    # Y_train[time,targets]

    regr = LassoCV(cv=5,max_iter=5000).fit(X,Y)
    Y_pred = regr.predict(X)
    
    r_squared=regr.score(X,Y)
    
    return regr,regr.coef_,r_squared,Y_pred

In [ ]:
def ridge(X,Y):

    # X_train[time,features]
    # Y_train[time,targets]

    regr = RidgeCV(cv=5).fit(X,Y)
    Y_pred = regr.predict(X)
    
    r_squared=regr.score(X,Y)
    
    return regr,regr.coef_,r_squared,Y_pred

In [ ]:
def tomsensomodel_regression(X,Y):
    
    model = Sequential()

    model.add(Dense(8, input_dim=X.shape[1],activation='tanh',
                kernel_initializer='he_normal',
                kernel_regularizer=regularizers.l1(0.02),
                bias_initializer='he_normal'))

    model.add(Dense(8, activation='tanh',
                kernel_initializer='he_normal',
                bias_initializer='he_normal'))

    model.add(Dense(2,name='output'))

    model.compile(optimizer=optimizers.Adam(lr=0.001), 
                  loss ='mean_squared_error', 
                  metrics = ['mse'])
        
    model.fit(X,Y,epochs=250, batch_size=50,verbose=0)
    
    return(model)

In [ ]:
data_path='../data/'
file='climateai_coding_challenge_data_v2.nc'

## Read Data

In [ ]:
ds=xr.open_dataset(data_path+file)

### Quick Check that Data looks reasonable

In [ ]:
ds

### Extract Dates to Match for features and target

In [ ]:
ds_trunc_precip=ds['precipitation'].sel(time_weather=slice('1957-12-28','2015-12-24'))
ds_trunc_temp=ds['temperature'].sel(time_weather=slice('1957-12-28','2015-12-24'))
ds_trunc_flow=ds['flow'].sel(time_flow=slice('1958-01-04','2015-12-31'))

print(ds_trunc_precip)
print(ds_trunc_temp)
print(ds_trunc_flow)

In [ ]:
plt.plot(ds_trunc_flow)

In [ ]:
ncols=3
nrows=3

fig = plt.figure()
for i,station in enumerate(ds['station_number']):
    plt.subplot(nrows,ncols,i+1)
    plt.plot(ds_trunc_precip.sel(station_number=station))
    
fig = plt.figure()
for i,station in enumerate(ds['station_number']):
    plt.subplot(nrows,ncols,i+1)
    plt.plot(ds_trunc_temp.sel(station_number=station))

## Define Features and Targets 
Features (`X`): precipitation, temperature, flow 
Target (`Y`): flow one week later

Y_dates=pd.date_range(start='1958-01-04',end='2015-12-31',freq='D')
Y_dates

print(Y_dates-pd.Timedelta(days=7))

X_dates=pd.date_range(start='1957-12-28',end='2015-12-24',freq='D')
X_dates

### Put Features and target into`numpy` arrays

In [ ]:
X=np.concatenate((ds_trunc_precip,ds_trunc_temp),axis=1)
print(ds_trunc_precip['time_weather'])
Y=ds_trunc_flow.where(ds_trunc_flow['time_flow'].isin(ds_trunc_precip['time_weather']+pd.Timedelta(days=7)), drop=True).values

In [ ]:
print('Check Features and Target Dimensions')
print('Features (X): ',X.shape)
print('Target (Y): ',Y.shape)

nsamples=X.shape[0]
nfeatures=X.shape[1]

print("Samples: ",nsamples)
print("features: ", nfeatures)

### Split Data into Training and Test

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.8)

ntrain=X_train.shape[0]
ntest=X_test.shape[0]

print('Training Size: ',ntrain)
print('Testing Size: ',ntest)

### Take a look at training data & Relationships between variables

#### Trend?

In [ ]:
plt.figure(figsize=(8.5,11))
y=np.arange(ntrain)

for i in range(nfeatures):

    plt.subplot(6,3,i+1)

    z = np.polyfit(y,X_train[:,i],1)
    p = np.poly1d(z)
    
    plt.plot(y,X_train[:,i])
    plt.plot(p(y),"r--")
    
    print("Check Stats: ", "Station: ",i, "Mean: ", X_train[:,i].mean(axis=0),"Var: ", X_train[:,i].var(axis=0))
    

#### How correlated are all the features and target with each other?

In [ ]:
tmp_data=np.hstack((X_train,np.expand_dims(Y_train, axis=1)))
d = pd.DataFrame(data=tmp_data)
corr=d.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(8.5,11))
sns.set(font_scale=1)
ax=sns.heatmap(corr,square=True,linewidths=0.5,fmt=" .2f", \
               annot=True,mask=mask,cmap='seismic', \
               vmin=-1,vmax=1,cbar=False,annot_kws={"size": 8})

### Seasonal Cycle & Anomalies

# Create an `xarray.Dataset` of temperature and precip training data

In [ ]:
ds_tmp1=xr.DataArray(X_train[:,0:9],
                    coords={'time':ds_trunc_precip['time_weather'][0:ntrain].values,
                            'stations': ds_trunc_precip['station_number'][0:9].values},
                    dims=['time','stations']).to_dataset(name='precipitation')
ds_tmp2=xr.DataArray(X_train[:,9::],
                    coords={'time':ds_trunc_precip['time_weather'][0:ntrain].values,
                            'stations': ds_trunc_precip['station_number'][0:9].values},
                    dims=['time','stations']).to_dataset(name='temperature')
ds_tmp=xr.merge([ds_tmp1,ds_tmp2])
ds_tmp

### Calculate the *noisy* climatology

In [ ]:
climo=ds_tmp.groupby('time.dayofyear').mean(dim='time')
climo

### Smooth to get the annual cycle

### Plot the Climatology for the temperature and precip stations

In [ ]:
for v in ['precipitation','temperature']:
    plt.figure(figsize=(8.5,11))
    for i in range(9):

        plt.subplot(3,3,i+1)
        plt.plot(climo['dayofyear'],climo[v][:,i])
        plt.title(v)


### Calculate Anomalies -- may use them for model

### Standardize the Data

In [ ]:
scaler_p=preprocessing.StandardScaler().fit(X_train[:,0:9])
scaler_t=preprocessing.StandardScaler().fit(X_train[:,9::])
X_scaled_p=scaler_p.transform(X_train[:,0:9])
X_scaled_t=scaler_t.transform(X_train[:,9::])
X_scaled=np.concatenate((X_scaled_p,X_scaled_t),axis=1)
scaler_y=preprocessing.StandardScaler().fit(Y_train.reshape(-1, 1))
Y_scaled=scaler_y.transform(Y_train.reshape(-1, 1))

### Linear Regression Models: LASSO

In [ ]:
regr,coeffs,rsq_train,Ypred=lasso(X_scaled,Y_scaled)
print('R^2 Train LASSO : ', rsq_train)
print(coeffs)

In [ ]:
regr,coeffs,rsq_train,Ypred=ridge(X_scaled,Y_scaled)
print('R^2 Train Ridge : ', rsq_train)
print(coeffs)

### Simple ANN based on Toms et al.

In [ ]:
tomsensomodel_regression(X,Y)

In [ ]:
nntoms=tomsensomodel_regression(X_train,Y_train)
#print('R^2 Train NN: ',rsq_train)
#print('R^2 Test NN: ',rsq_test)